# Model Evaluation of NeoPrecis-Immuno

This notebook is for evaluating immunogenicity prediction. 

The data is presented in Table S2-3, and the results are shown in Figure 2-3.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from venn import venn
from imm_utils import (
    Bootstrapping,
    BootstrappingPN,
    Performance,
    TwoPerfBarPlot,
    FilterNCI
)

dpi = 600

In [ ]:
# path

triplet_file = 'SuppData1.csv' # fill in the path of Supplementary Data 1
cedar_file = 'SuppData2.csv' # fill in the path of Supplementary Data 2
nci_file = 'SuppData3.csv'  # fill in the path of Supplementary Data 3
out_dir = './'
"""
work_dir = ''
triplet_file = f'{work_dir}/manuscript/tables/triplet.round.csv'
cedar_file = f'{work_dir}/manuscript/tables/cedar.round.csv'
nci_file = f'{work_dir}/manuscript/tables/nci.round.csv'
out_dir = f'{work_dir}/manuscript/plots/np_immuno_eval'
"""
nci_allele_file = '../data/nci_alleles.csv'

savefig = True

In [ ]:
# load

cedar_df = pd.read_csv(cedar_file)
nci_df = pd.read_csv(nci_file)
triplet_df = pd.read_csv(triplet_file)
nci_allele_df = pd.read_csv(nci_allele_file, index_col=0)

if not os.path.isdir(out_dir):
    os.mkdir(out_dir)

# model color map
models = ['NP-Immuno', 'NetMHCpan', 'DeepNeo', 'PRIME', 'ICERFIRE']
metric_color_dict = {models[i]: sns.color_palette('pastel')[i] for i in range(len(models))}
metric_color_dict['PHBR'] = metric_color_dict['NetMHCpan']
metric_color_dict['NetMHCIIpan'] = metric_color_dict['NetMHCpan']

### Validation on the CEDAR immunogenicity dataset

#### all testing

In [ ]:
# evaluation with bootstrapping (only MHC-I)

x_col_dict = {
    'MT_netMHCpan_rank': 'less',
    'PRIME': 'less',
    'ICERFIRE': 'less',
    'DeepNeo': 'greater',
    'NeoPrecis-Immuno': 'greater'
}
y_col = 'label'

# testing set
# select rows where none of the in_* flags are True and none are NA
mask_present_in_others = (~cedar_df[['in_prime', 'in_icerfire', 'in_deepneo']].any(axis=1)) & cedar_df[['in_prime', 'in_icerfire', 'in_deepneo']].notna().all(axis=1)
test_df = cedar_df[mask_present_in_others]  # not present in other predictors' training set
test_df = test_df[~test_df['dataset'].isin(['train', 'valid'])] # not in our training set
test_df = test_df[test_df['MHC']=='I'] # only MHC-I
test_df = test_df.dropna(subset=['PRIME', 'ICERFIRE', 'DeepNeo'])
print('#Samples =', test_df.shape[0])

# result
perf_df = Performance(test_df, x_col_dict, y_col, fillna=False) # performance
print(perf_df)

# bootstrapping
bstp_df = Bootstrapping(test_df, x_col_dict, y_col, fillna=False, n_iter=1000)

In [ ]:
# plot - AUROC

metric = 'AUROC'

# df
plot_df = bstp_df.copy()
plot_df['Model'] = plot_df['Model'].replace({'MT_netMHCpan_rank': 'NetMHCpan', 'NeoPrecis-Immuno': 'NP-Immuno'})

# plot
order_list = ['NP-Immuno', 'NetMHCpan', 'DeepNeo', 'PRIME', 'ICERFIRE']
fig, ax = plt.subplots(1, 1, figsize=(3, 3), dpi=dpi)
g = sns.boxplot(data=plot_df, x='Model', y=metric, order=order_list, ax=ax)
ax.tick_params(axis='x', rotation=60)
_ = ax.set_xlabel('')

# color
for i, patch in enumerate(g.patches):
    patch.set_facecolor(metric_color_dict[order_list[i]])

# save
fig.tight_layout()
if savefig:
    fig.savefig(f'{out_dir}/cedar_test_auroc.png')

In [ ]:
# plot - AUPRC

metric = 'AUPRC'

# df
plot_df = bstp_df.copy()
plot_df['Model'] = plot_df['Model'].replace({'MT_netMHCpan_rank': 'NetMHCpan', 'NeoPrecis-Immuno': 'NP-Immuno'})

# plot
order_list = ['NP-Immuno', 'NetMHCpan', 'DeepNeo', 'PRIME', 'ICERFIRE']
fig, ax = plt.subplots(1, 1, figsize=(3, 3), dpi=dpi)
g = sns.boxplot(data=plot_df, x='Model', y=metric, order=order_list, ax=ax)
ax.tick_params(axis='x', rotation=60)
_ = ax.set_xlabel('')

# color
for i, patch in enumerate(g.patches):
    patch.set_facecolor(metric_color_dict[order_list[i]])

# save
fig.tight_layout()
if savefig:
    fig.savefig(f'{out_dir}/cedar_test_auprc.png')

#### positive : negative = 1 : 3

In [ ]:
# filtering
test_df = cedar_df[~(cedar_df[['in_prime', 'in_icerfire', 'in_deepneo']]).all(axis=1)] # not present in other predictors' training set
test_df = test_df[~test_df['dataset'].isin(['train', 'valid'])] # not in our training set
test_df = test_df[test_df['MHC']=='I'] # only MHC-I
test_df = test_df.dropna(subset=['PRIME', 'ICERFIRE', 'DeepNeo'])
test_df['length'] = test_df['WT_pept'].apply(lambda x: len(x))

In [ ]:
# bootstrapping
pred_cols = ['NeoPrecis-Immuno', 'PRIME', 'ICERFIRE', 'DeepNeo', 'MT_netMHCpan_rank']
pred_col_dict = {col: ('less' if col in ['MT_netMHCpan_rank', 'PRIME', 'ICERFIRE'] else 'greater') for col in pred_cols}
label_col = 'label'
p_n_ratio = 0.333
result_df = BootstrappingPN(test_df, pred_col_dict, label_col, p_n_ratio=p_n_ratio, fillna=True, n_iter=1000)

In [ ]:
# plot
plot_df = result_df.copy()
plot_df['Model'] = plot_df['Model'].replace({'MT_netMHCpan_rank': 'NetMHCpan', 'NeoPrecis-Immuno': 'NP-Immuno'})
order_list = ['NP-Immuno', 'NetMHCpan', 'DeepNeo', 'PRIME', 'ICERFIRE']

fig, ax = plt.subplots(1, 2, figsize=(7,3), dpi=dpi)
sns.boxplot(data=plot_df, x='Model', y='AUROC', hue='Model', order=order_list, palette=metric_color_dict, ax=ax[0])
sns.boxplot(data=plot_df, x='Model', y='AUPRC', hue='Model', order=order_list, palette=metric_color_dict, ax=ax[1])
_ = ax[0].set_xlabel('')
_ = ax[1].set_xlabel('')
ax[0].tick_params(axis='x', labelrotation=45)
ax[1].tick_params(axis='x', labelrotation=45)

fig.tight_layout()
fig.savefig(f'{out_dir}/cedar_test_balance.png')

### Validation on the NCI dataset

In [ ]:
# filtered by abundance and presentation
def NCI_comparison(mut_df, mhc, x_cols, y_col, dropna=False):
    # x_cols
    x_col_dict = {f'{col}-{mhc}': ('less' if col in ['PHBR', 'PRIME', 'ICERFIRE'] else 'greater') for col in x_cols}
    x_col_dict = {k:v for k,v in x_col_dict.items() if k in mut_df.columns}

    # filtering
    print('#Samples =', mut_df.shape[0])
    filter_mut_df = mut_df[(mut_df['DNA_AF']>0) & (mut_df['RNA_AF']>0) & (mut_df['RNA_EXP_QRT']>1)] # filter by expression
    print('Filtered by abundance =', filter_mut_df.shape[0])
    filter_mut_df = filter_mut_df[filter_mut_df['Consequence']=='missense_variant'] # focus on substitutions
    print('Filtered by SNVs =', filter_mut_df.shape[0])
    filter_mut_df = filter_mut_df.dropna(subset=[f'PHBR-{mhc}']) # drop invalid mutations
    print('Filtered by valid PHBR =', filter_mut_df.shape[0])
    filter_mut_df = filter_mut_df[filter_mut_df[f'Robustness-{mhc}']>0]
    print('Filtered by robustness>0 =', filter_mut_df.shape[0])
    if dropna:
        cols = list(x_col_dict.keys()) + [y_col,]
        filter_mut_df = filter_mut_df.dropna(subset=cols) # drop NA
        print('Filtered by NA =', filter_mut_df.shape[0])

    # validation
    results = Performance(filter_mut_df, x_col_dict, y_col, fillna=True)

    return filter_mut_df, results

In [ ]:
### comparing with other methods
# drop NA

methods = ['NP-Immuno', 'PHBR', 'DeepNeo', 'PRIME', 'ICERFIRE']

print('===MHCI===')
mhci_filter_mut_df, mhci_results = NCI_comparison(nci_df, 'I', methods, 'CD8', dropna=True) # drop NA to ensure all samples with predictions
print('===MHCII===')
mhcii_filter_mut_df, mhcii_results = NCI_comparison(nci_df, 'II', methods, 'CD4', dropna=True)

# rename PPV@k (k=#positives)
mhci_results = mhci_results.rename(columns={f'PPV@{mhci_filter_mut_df["CD8"].sum()}': 'PPV@k'})
mhcii_results = mhcii_results.rename(columns={f'PPV@{mhcii_filter_mut_df["CD4"].sum()}': 'PPV@k'})

# plot
figfile = f'{out_dir}/nci_test_auroc.png' if savefig else None
TwoPerfBarPlot(mhci_results, mhcii_results, methods, 'AUROC', palette=metric_color_dict, hue_order=methods, figsize=(4,3), figfile=figfile)

fig, ax = plt.subplots(1, 2, figsize=(7, 3), dpi=dpi)
TwoPerfBarPlot(mhci_results, mhcii_results, methods, 'AUPRC', palette=metric_color_dict, hue_order=methods, ax=ax[0])
TwoPerfBarPlot(mhci_results, mhcii_results, methods, 'PPV@k', palette=metric_color_dict, hue_order=methods, ax=ax[1])
ax[1].legend_.remove()
sns.move_legend(ax[0], loc='center right', bbox_to_anchor=(-.3, 0.5))
fig.tight_layout()
if savefig:
    fig.savefig(f'{out_dir}/nci_test.png')

In [ ]:
### component contribution
# fill NA with 0

methods = ['BLOSUMDist', 'PMBECDist', 'SubDist', 'SubPosDist', 'GeoDist', 'CRD']
metric = 'AUROC'

print('===MHCI===')
mhci_filter_mut_df, mhci_results = NCI_comparison(nci_df, 'I', methods, 'CD8', dropna=False)
print('===MHCII===')
mhcii_filter_mut_df, mhcii_results = NCI_comparison(nci_df, 'II', methods, 'CD4', dropna=False)

figfile = f'{out_dir}/nci_components.png' if savefig else None
TwoPerfBarPlot(mhci_results, mhcii_results, methods, metric, palette='pastel', figsize=(7,3), annot=True, figfile=figfile)

In [ ]:
### component correlation

cols = ['BLOSUMDist', 'PMBECDist', 'SubDist', 'SubPosDist', 'GeoDist', 'CRD']
mhci_cols = [f'{col}-I' for col in cols]
mhcii_cols = [f'{col}-II' for col in cols]

# corr
mhci_corr_df = mhci_filter_mut_df[mhci_cols].corr()
mhcii_corr_df = mhcii_filter_mut_df[mhcii_cols].corr()

# plot
fig, ax = plt.subplots(1, 2, figsize=(6, 3), dpi=dpi, gridspec_kw={'width_ratios': [0.45, 0.55]})
sns.heatmap(mhci_corr_df, cmap='Blues', ax=ax[0], vmin=0, vmax=1, cbar=False, annot=True)
sns.heatmap(mhcii_corr_df, cmap='Blues', ax=ax[1], vmin=0, vmax=1, annot=True)
_ = ax[0].set_xticklabels(cols)
_ = ax[1].set_xticklabels(cols)
_ = ax[0].set_yticklabels(cols)
_ = ax[1].set_yticklabels('')
_ = ax[1].set_yticks([])
_ = ax[0].set_title('MHC-I')
_ = ax[1].set_title('MHC-II')

fig.tight_layout()
if savefig:
    fig.savefig(f'{out_dir}/nci_component_corr.png')

## Generalizability analysis

#### TCR coverage

In [ ]:
### two levels: pMHC and a.a.

group_col_dict = {
    'pMHC level': ['mhc', 'allele', 'anchor_peptide', 'positive_peptide', 'positive_mutPos', 'positive_ref', 'positive_alt'],
    'a.a. level': ['mhc', 'allele', 'positive_mutPos', 'positive_ref', 'positive_alt']
}

fig, ax = plt.subplots(1, 2, figsize=(10, 3), dpi=dpi)

count_name = 'CDR3 count'
for i, (group_name, group_cols) in enumerate(group_col_dict.items()):
    count_df = triplet_df.groupby(group_cols).size() / 10
    count_df = count_df.rename(count_name).reset_index()
    count_df[count_name] = (count_df[count_name]+0.5).astype(int)
    count_df[f'log({count_name})'] = count_df[count_name].apply(lambda x: np.log10(x))
    sns.ecdfplot(data=count_df, x=f'log({count_name})', hue='mhc', stat='proportion', ax=ax[i])
    sns.move_legend(ax[i], loc='lower right')
    ax[i].set_title(group_name)

fig.tight_layout()
if savefig:
    fig.savefig(f'{out_dir}/crd_tcr_count.png')

#### MHC coverage

In [ ]:
### allele counts

allele_dict = {
    'mhci':{
        'triplet': set(triplet_df.loc[triplet_df['mhc']=='mhci', 'allele'].unique()),
        'cedar_train': set(cedar_df.loc[(cedar_df['MHC']=='I') & (cedar_df['dataset']!='test'), 'allele'].unique()),
        'cedar_test': set(cedar_df.loc[(cedar_df['MHC']=='I') & (cedar_df['dataset']=='test'), 'allele'].unique()),
    },
    'mhcii':{
        'triplet': set(triplet_df.loc[triplet_df['mhc']=='mhcii', 'allele'].unique()),
        'cedar_train': set(cedar_df.loc[(cedar_df['MHC']=='II') & (cedar_df['dataset']!='test'), 'allele'].unique()),
        'cedar_test': set(cedar_df.loc[(cedar_df['MHC']=='II') & (cedar_df['dataset']=='test'), 'allele'].unique()),
    },
}

# plot
fig, ax = plt.subplots(2, 1, figsize=(4,6), dpi=dpi)
venn(allele_dict['mhci'], ax=ax[0], fontsize=10)
venn(allele_dict['mhcii'], ax=ax[1], fontsize=10)
ax[0].set_title('MHC-I')
ax[1].set_title('MHC-II')
ax[0].legend_.remove()
sns.move_legend(ax[1], loc='center right', bbox_to_anchor=(1.3,1.1))

fig.tight_layout()
if savefig:
    fig.savefig(f'{out_dir}/crd_allele_count.png')

#### unobserved alleles

In [ ]:
### number of unobserved alleles of each patient (homozygous will count 2)

mhci_obs_alleles = allele_dict['mhci']['cedar_train'] | allele_dict['mhci']['triplet']
mhcii_obs_alleles = allele_dict['mhcii']['cedar_train'] | allele_dict['mhcii']['triplet']

mhci_unobs_count_dict, mhcii_unobs_count_dict = dict(), dict()
for idx, row in nci_allele_df.iterrows():
    # alleles
    mhci_alleles = row[['A_1', 'A_2', 'B_1', 'B_2', 'C_1', 'C_2']].tolist()
    mhcii_alleles = row[['DRB_1', 'DRB_2']].tolist()
    for s in ['Q', 'P']:
        if row[f'D{s}A_1'] is not None:
            for i in [1, 2]:
                for j in [1, 2]:
                    allele_a = row[f'D{s}A_{i}']
                    allele_b = row[f'D{s}B_{j}']
                    allele = f'{allele_a}_{allele_b}'
                    mhcii_alleles.append(allele)

    # counts
    mhci_unobs_count = len([a for a in mhci_alleles if a not in mhci_obs_alleles])
    mhcii_unobs_count = len([a for a in mhcii_alleles if a not in mhcii_obs_alleles])
    id_name = int(idx[6:])
    mhci_unobs_count_dict[id_name] = mhci_unobs_count
    mhcii_unobs_count_dict[id_name] = mhcii_unobs_count

# add observed allele count
nci_df['mhci_unobs_alleles'] = nci_df['Patient'].apply(lambda x: mhci_unobs_count_dict[x])
nci_df['mhcii_unobs_alleles'] = nci_df['Patient'].apply(lambda x: mhcii_unobs_count_dict[x])

In [ ]:
### proportion of unobserved alleles

mhc_label_dict = {'i': 'CD8', 'ii': 'CD4'}
pred_cols = ['NP-Immuno', 'PRIME', 'ICERFIRE', 'DeepNeo']

fig, ax = plt.subplots(1, 2, figsize=(6, 3), dpi=dpi)
for i, (mhc, label) in enumerate(mhc_label_dict.items()):
    filter_df = FilterNCI(nci_df, mhc.upper(), pred_cols, label, dropna=True)
    sns.ecdfplot(data=filter_df, x=f'mhc{mhc}_unobs_alleles', stat='proportion', ax=ax[i])
    ax[i].xaxis.set_major_locator(plt.MaxNLocator(integer=True))
    _ = ax[i].set_title(f'MHC{mhc.upper()}')
    _ = ax[i].set_xlabel('#unobserved alleles')

fig.tight_layout()
if savefig:
    fig.savefig(f'{out_dir}/nci_unobs_count.png')

In [ ]:
### performance: fewer vs. more unobserved alleles

### evaluating two groups (fewer vs. more unobserved alleles)

mhc_label_dict = {'i': 'CD8', 'ii': 'CD4'}
mhc_count_thrs_dict = {'i': 0, 'ii': 5}
pred_cols = ['NP-Immuno', 'PRIME', 'ICERFIRE', 'DeepNeo']
bootstrapping_niters = 1000

fig, ax = plt.subplots(1, 2, figsize=(6,3), dpi=dpi)

for i, (mhc, label) in enumerate(mhc_label_dict.items()):
    result_df = list()
    
    # filter df
    filter_df = FilterNCI(nci_df, mhc.upper(), pred_cols, label, dropna=True)

    # pred cols
    pred_col_dict = {f'{col}-{mhc.upper()}': ('less' if col in ['PHBR', 'PRIME', 'ICERFIRE'] else 'greater') for col in pred_cols}
    pred_col_dict = {k:v for k,v in pred_col_dict.items() if k in filter_df.columns}

    # two groups (obs vs. unobs)
    count_thrs = mhc_count_thrs_dict[mhc]
    obs_df = filter_df[filter_df[f'mhc{mhc}_unobs_alleles']<=count_thrs]
    unobs_df = filter_df[filter_df[f'mhc{mhc}_unobs_alleles']>count_thrs]

    tmp_result_df = Bootstrapping(obs_df, pred_col_dict, label, fillna=False, n_iter=bootstrapping_niters)
    tmp_result_df['MHC'] = mhc.upper()
    tmp_result_df['group'] = f'#unobs. <={count_thrs}\n(n={obs_df.shape[0]})'
    result_df.append(tmp_result_df)

    tmp_result_df = Bootstrapping(unobs_df, pred_col_dict, label, fillna=False, n_iter=bootstrapping_niters)
    tmp_result_df['MHC'] = mhc.upper()
    tmp_result_df['group'] = f'#unobs. >{count_thrs}\n(n={unobs_df.shape[0]})'
    result_df.append(tmp_result_df)

    result_df = pd.concat(result_df, axis=0, ignore_index=True)

    # perf plot
    result_df['Model'] = result_df['Model'].apply(lambda x: '-'.join(x.split('-')[:-1]))
    sns.boxplot(data=result_df, x='group', y='AUROC', hue='Model', palette=metric_color_dict, ax=ax[i])
    _ = ax[i].set_xlabel(f'MHC{mhc.upper()}')
    ax[i].legend_.remove()

handles, labels = ax[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 1), ncol=len(labels))


fig.tight_layout()
fig.subplots_adjust(top=0.87)
if savefig:
    fig.savefig(f'{out_dir}/nci_unobs_perf.png')